In [1]:
import sys
sys.path.append("/media/hdd/viscent/SR-UNet")
from utils.inference_utils import *

In [2]:
image_tensor, target_tensor = load_image(dataset='hcp')

In [7]:
image = sitk.GetImageFromArray(image_tensor[0, 0].detach().cpu().numpy()*4096)
sitk.WriteImage(image, 'Input_HCP.nii.gz')

In [6]:
image = sitk.GetImageFromArray(target_tensor[0, 0].detach().cpu().numpy()*4096)
sitk.WriteImage(image, 'Template_HCP.nii.gz')